# Notebook 01: Foundation & AWS Setup

## Learning Objectives
- Set up AWS environment for AgentCore
- Understand AgentCore architecture
- Validate development environment
- Configure basic authentication

## Prerequisites
- AWS account with appropriate permissions
- Python 3.10+ with uv package manager
- API keys for external services

**⚠️ IMPORTANT: Run `./setup.sh` from project root before starting this notebook!**

## Step 1: Environment Setup

In [ ]:
# Verify environment setup
import os
import sys

# Check if we're in the right directory and environment is set up
if not os.path.exists('../../pyproject.toml'):
    print('❌ Please run ./setup.sh from project root first!')
    sys.exit(1)

print(f'✅ Working directory: {os.getcwd()}')
print('✅ Environment setup verified')

In [ ]:
import os

os.environ['AWS_REGION'] = 'us-east-1'

# APPROACH A: Use credentials
# os.environ['AWS_ACCESS_KEY_ID'] = 'your_access_key'
# os.environ['AWS_SECRET_ACCESS_KEY'] = 'your_secret_key'
# os.environ['AWS_SESSION_TOKEN'] = "your_session_token"

# APPROACH B: Use AWS SSO profile
#os.environ['AWS_PROFILE'] = 'your_profile'
# Remove any existing credential env vars to force profile usage
#for key in ['AWS_ACCESS_KEY_ID', 'AWS_SECRET_ACCESS_KEY', 'AWS_SESSION_TOKEN']:
#    os.environ.pop(key, None)

os.environ['AWS_REGION'] = 'us-east-1'

print("✅ AWS Profile set. Please restart kernel and run all cells.")

In [ ]:
import boto3
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Verify AWS credentials
session = boto3.Session()
credentials = session.get_credentials()
print(f"AWS Access Key: {credentials.access_key[:8]}...")
print(f"AWS Region: {session.region_name or 'us-west-2'}")

## Step 2: API Keys Validation

In [ ]:
# Validate API keys
api_keys = {
    "AVIATIONSTACK_API_KEY": os.getenv("AVIATIONSTACK_API_KEY"),
    "OPENWEATHERMAP_API_KEY": os.getenv("OPENWEATHERMAP_API_KEY"),
    "EXCHANGERATE_API_KEY": os.getenv("EXCHANGERATE_API_KEY")
}

print("🔑 API Key Status:")
for key_name, key_value in api_keys.items():
    if key_value:
        print(f"✅ {key_name}: {'*' * 8}{key_value[-4:]}")
    else:
        print(f"❌ {key_name}: Not configured")

missing_keys = [k for k, v in api_keys.items() if not v]
if missing_keys:
    print(f"\n⚠️ Missing API keys: {', '.join(missing_keys)}")
    print("Please configure these before proceeding.")
else:
    print("\n✅ All API keys configured!")

## Step 3: Environment Validation

In [ ]:
# Comprehensive environment check
import sys
import subprocess

def check_environment():
    checks = []
    
    # Python version
    python_version = sys.version_info
    if python_version >= (3, 9):
        checks.append(("✅", f"Python {python_version.major}.{python_version.minor}"))
    else:
        checks.append(("❌", f"Python {python_version.major}.{python_version.minor} (need 3.9+)"))
    
    # uv package manager
    try:
        result = subprocess.run(["uv", "--version"], capture_output=True, text=True)
        if result.returncode == 0:
            checks.append(("✅", f"uv {result.stdout.strip()}"))
        else:
            checks.append(("❌", "uv not found"))
    except FileNotFoundError:
        checks.append(("❌", "uv not installed"))
    
    # AWS credentials
    try:
        sts = boto3.client('sts')
        identity = sts.get_caller_identity()
        checks.append(("✅", f"AWS Account: {identity['Account']}"))
    except Exception as e:
        checks.append(("❌", f"AWS credentials: {str(e)}"))
    
    # Bedrock access
    try:
        bedrock = boto3.client('bedrock', region_name='us-west-2')
        bedrock.list_foundation_models()
        checks.append(("✅", "Bedrock access"))
    except Exception as e:
        checks.append(("❌", f"Bedrock access: {str(e)}"))
    
    return checks

print("🔍 Environment Validation:")
validation_results = check_environment()
for status, message in validation_results:
    print(f"{status} {message}")

# Summary
passed = sum(1 for status, _ in validation_results if status == "✅")
total = len(validation_results)
print(f"\n📊 Validation Summary: {passed}/{total} checks passed")

if passed == total:
    print("🎉 Environment ready! Proceed to Notebook 02.")
else:
    print("⚠️ Please fix the issues above before continuing.")

## Next Steps

✅ **Completed in this notebook:**
- AWS environment setup and validation
- AgentCore introduction and concepts
- API keys configuration check
- Development environment verification

➡️ **Next: Notebook 02 - Runtime Setup**
- Configure AgentCore Runtime
- Create basic conversational agent
- Implement multi-turn dialogue
- Test conversation flow